# Module 12 — Model Evaluation, Explainability & Debugging (v2)

Updated notebook with clear sections and lightweight explainability examples suitable for classroom demos. Includes:

- classification metrics (confusion matrix, ROC, PR)
- SHAP for tabular models (small sample + bar plot)
- Grad-CAM for CNNs (single-image overlay)
- debugging checklist and classroom exercises

This version uses small datasets and is careful to keep computations light for Colab demos.

## 1 — Setup (install packages and imports)

In [ ]:
# Install minimal required packages
!pip -q install -U tensorflow scikit-learn shap matplotlib --quiet

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve, precision_recall_curve
import shap
print('TF version:', tf.__version__)
print('SHAP version:', shap.__version__)


## 2 — Tabular baseline model & evaluation (RandomForest)

In [ ]:
# Create a small synthetic dataset
from sklearn.model_selection import train_test_split
X, y = make_classification(n_samples=300, n_features=8, n_informative=4, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train RandomForest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Predict & metrics
y_pred = rf.predict(X_test)
y_proba = rf.predict_proba(X_test)[:,1]
print('Classification report:\n')
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print('Confusion matrix:\n', cm)

# ROC & PR
roc_auc = roc_auc_score(y_test, y_proba)
fpr, tpr, _ = roc_curve(y_test, y_proba)
prec, rec, _ = precision_recall_curve(y_test, y_proba)

plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(fpr, tpr, label=f'ROC AUC={roc_auc:.2f}')
plt.xlabel('FPR'); plt.ylabel('TPR'); plt.title('ROC Curve'); plt.legend()
plt.subplot(1,2,2)
plt.plot(rec, prec); plt.xlabel('Recall'); plt.ylabel('Precision'); plt.title('Precision-Recall Curve')
plt.show()


## 3 — SHAP explanation (lightweight)

In [ ]:
# Use a small background dataset to keep SHAP fast
background = X_train[np.random.choice(X_train.shape[0], size=min(50, X_train.shape[0]), replace=False)]
explainer = shap.Explainer(rf.predict_proba, background)
# explain a few test instances only
to_explain = X_test[:10]
shap_values = explainer(to_explain)

# Use a compact bar plot for feature importance across the explained samples
shap.plots.bar(shap_values)


## 4 — Grad-CAM for CNNs (Keras) — single-image demo on CIFAR-10 subset

In [ ]:
# Build and train a tiny CNN on a small CIFAR-10 subset, then compute Grad-CAM for one image
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import layers, models
import numpy as np

(x_train,y_train),(x_test,y_test) = cifar10.load_data()
x_train = x_train[:1500].astype('float32')/255.0; y_train = y_train[:1500]
x_test = x_test[:300].astype('float32')/255.0; y_test = y_test[:300]

def tiny_cnn():
    inputs = layers.Input((32,32,3))
    x = layers.Conv2D(16,3,activation='relu', name='conv1')(inputs)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(32,3,activation='relu', name='conv2')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation='relu')(x)
    outputs = layers.Dense(10, activation='softmax')(x)
    return models.Model(inputs, outputs)

model = tiny_cnn()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=3, batch_size=64, validation_split=0.1, verbose=1)

# Pick a sample image
idx = 5
img = x_test[idx:idx+1]
probs = model.predict(img)
pred_class = np.argmax(probs[0])
print('Predicted class:', pred_class)

# Grad-CAM
last_conv_layer_name = 'conv2'
grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(last_conv_layer_name).output, model.output])
with tf.GradientTape() as tape:
    conv_outputs, predictions = grad_model(img)
    loss = predictions[:, pred_class]

grads = tape.gradient(loss, conv_outputs)[0]
pooled_grads = tf.reduce_mean(grads, axis=(0,1))
conv_outputs = conv_outputs[0].numpy()
pooled_grads = pooled_grads.numpy()

# Weight the channels by pooled grads
for i in range(pooled_grads.shape[-1]):
    conv_outputs[:,:,i] *= pooled_grads[i]
heatmap = np.mean(conv_outputs, axis=-1)
heatmap = np.maximum(heatmap, 0)
heatmap /= (np.max(heatmap) + 1e-8)

# resize heatmap to 32x32 and overlay
import cv2
heatmap_resized = cv2.resize(heatmap, (32,32))
heatmap_uint = np.uint8(255 * heatmap_resized)
heatmap_color = cv2.applyColorMap(heatmap_uint, cv2.COLORMAP_JET)
superimposed = heatmap_color * 0.4 + np.uint8(img[0]*255)

plt.figure(figsize=(8,4))
plt.subplot(1,2,1); plt.imshow(img[0]); plt.title('Original'); plt.axis('off')
plt.subplot(1,2,2); plt.imshow(superimposed.astype('uint8')); plt.title('Grad-CAM'); plt.axis('off')
plt.show()


## 5 — Debugging checklist & classroom exercises

**Checklist**
- Data leakage: check splits and ensure no overlap.
- Label noise: sample and inspect potentially wrong labels.
- Class imbalance: compute class distribution; consider class weights or resampling.
- Learning rate and optimization: try different optimizers or learning rate schedules.
- Overfitting: compare train/val curves, use augmentation or regularization.

**In-class exercises**
1. Run SHAP on a misclassified example and interpret which features pushed the prediction.
2. Use Grad-CAM on several misclassified images and discuss what the model focuses on.
3. Introduce a synthetic label noise (flip 5% labels) and observe effect on model performance.
